In [1]:
# 什么是基础用户？
# 比如整个时间探测器测量了一周的数据，我们将时间进行切片，3，2，2，其中3作为基础数据，
# 将3时间长度的数据，作为基础数据，然后用2，2出现的用户，和3的用户进行相与，相与的用户，才是基础用户。
# 其他的用户都是要判断新入网，新离网的。

思路：
- 这里假设数据有9天，先假设三天的数据为基础数据。然后用后面3天的数据去检测前三天的数据
- 然后用后面3天的数据检测前6天的数据，重新给前面的标签进行标记。
- 后面3天的数据，除去一些基础用户，其他都是属于新入网用户。
- 然后前面9天的数据，又由后面的检测数据进行标记

# 导入模块

In [2]:
import pandas as pd
import os
from os import path

import warnings

warnings.filterwarnings('ignore')

In [18]:
import random

In [72]:
from datetime import date
from datetime import time

# 定义函数

In [8]:
def get_data(file_name):
    '''
    传入的是文件路径
    读取并对数据进行处理
    '''
    file_extension = path.splitext(file_name)[1]
    if file_extension == '.csv':
        data = pd.read_csv(file_name, encoding='gbk')
    elif file_extension == '.xlsx':
        data = pd.read_excel(file_name)
    drop_col = ['imei_', 'area_', 'msisdn_']
    data.drop(drop_col, axis=1, inplace=True)
    col = data.columns
    for each in col:
        if data[each].dtype == object:
            data[each] = data[each].str.replace('\t', '')
    print('data processed successfully')
    return data

In [310]:
def trans_to_time(data):
    data.index = pd.to_datetime(data.update_time_)
    return data

In [319]:
def get_status(data):
    # 完整调试
    year = data.index.year.min()
    month = data.index.month.min()
    day = data.index.day.min() # 本来不加min返回的是所有年份
    time1 = datetime.datetime(year, month, day)

    time_window = datetime.timedelta(days=window) # 这样可以直接和时间数据相加

    time2 = (time1 + time_window)
    # 将时间类型数据转换成字符串
    stime1, stime2 = time1.strftime(format='%Y-%m-%d %H:%M:%S'), time2.strftime(format='%Y-%m-%d %H:%M:%S')

    # data.imsi_.loc[stime1:stime2]
    d = {}
    base_user = set(data.imsi_.loc[stime1:stime2])
    while time1.day + window <= data.index.day.max():
        time1, time2 = time2, time2 + time_window
        stime1, stime2 = time1.strftime(format='%Y-%m-%d %H:%M:%S'), time2.strftime(format='%Y-%m-%d %H:%M:%S')
        user = set(data.imsi_.loc[stime1:stime2]) # 新用户
        new_leaver = base_user - user # 注意集合运算的顺序 1 新离网
        permanent = base_user & user # 2 常驻用户
        new_commer = user - base_user # 0 新入网
        base_user.update(user)
        d.update(dict(zip(new_leaver, len(new_leaver)*[1])))
        d.update(dict(zip(new_commer, len(new_commer)*[0])))
        d.update(dict(zip(permanent, len(permanent)*[2])))
    data['status'] = data.imsi_.map(d)
    return data

In [320]:
def main():
    os.listdir('../Project Data/')
    os.chdir('../Project Data/')
    data = get_data('20190524-001.csv')
    data = trans_to_time(data)
    data = get_status(data)
    

# 代码运行

data processed successfully
                               imsi_         update_time_  lac_  cellid_  \
update_time_                                                               
2019-05-08 20:09:29  460110413044304  2019-05-08 20:09:29     0        0   
2019-05-08 20:09:41  460110413042577  2019-05-08 20:09:41     0        0   
2019-05-08 20:10:14  460110425023300  2019-05-08 20:10:14     0        0   
2019-05-08 20:10:40  460110714141865  2019-05-08 20:10:40     0        0   
2019-05-08 20:08:55  460016692088409  2019-05-08 20:08:55     0        0   

                    status_ ap_type_  device_id_    eventid_  status  
update_time_                                                          
2019-05-08 20:09:29      拒绝  FDD-LTE  DX-SZSC001  Normal Lau       2  
2019-05-08 20:09:41      拒绝  FDD-LTE  DX-SZSC001  Normal Lau       1  
2019-05-08 20:10:14      拒绝  FDD-LTE  DX-SZSC001  Normal Lau       1  
2019-05-08 20:10:40      拒绝  FDD-LTE  DX-SZSC001  Normal Lau       2  
2019-05-08 20

# 代码调试

In [9]:
os.listdir('../Project Data/')

['4月',
 '20190529',
 '20190529-4G电围.xls',
 '.DS_Store',
 '分析数据',
 '3月',
 'data_eda.ipynb',
 'same',
 '20190529-001.csv',
 '20190424-001.xlsx',
 '20190524-001.csv',
 '20190529-4G_resave.csv',
 'test.py',
 '20190422-灵瞳感知器.xls',
 '最新数据',
 '050823数据表']

In [10]:
os.chdir('../Project Data/')

In [160]:
data = get_data('20190524-001.csv')

data processed successfully


In [13]:
data.head()

,imsi_,update_time_,lac_,cellid_,status_,ap_type_,device_id_,eventid_
0,460110413044304,2019-05-08 20:09:29,0,0,拒绝,FDD-LTE,DX-SZSC001,Normal Lau
1,460110413042577,2019-05-08 20:09:41,0,0,拒绝,FDD-LTE,DX-SZSC001,Normal Lau
2,460110425023300,2019-05-08 20:10:14,0,0,拒绝,FDD-LTE,DX-SZSC001,Normal Lau
3,460110714141865,2019-05-08 20:10:40,0,0,拒绝,FDD-LTE,DX-SZSC001,Normal Lau
4,460016692088409,2019-05-08 20:08:55,0,0,拒绝,FDD-LTE,LT-SZSC001,Normal Lau


In [161]:
data.index = pd.to_datetime(data.update_time_)

In [17]:
data.index.max() - data.index.min() # 时间周期是16天

Timedelta('16 days 02:30:09')

In [19]:
# 应该随机选取天数。

In [30]:
for each in range(3): # 想要生成随机不重复的，还得使用kfold。这样会把所有天数都遍历一遍
    print(random.sample([1,2,3,4,5,6],2))

[4, 3]
[3, 2]
[2, 6]


In [45]:
t = set(range(5))

In [44]:
t

set()

In [39]:
random.sample(t,2)

[4, 0]

In [46]:
# 可以用这样的方式实现类似kfold的函数
# 如果不是整数，则会报错

for i in range(3):
    try:
        temp = random.sample(t, 2)
    except ValueError:
        temp = random.sample(t, len(t))
    t -= set(temp)
    print(temp)

[3, 0]
[1, 2]
[4]


In [47]:
# 我们以天为单位，随机选取两天作为基础数据

te = set(data.index.day)

In [48]:
te

{8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24}

In [ ]:
te

In [50]:
# 测试集合的一些方法

s = set()
s.add(1)

In [51]:
s

{1}

In [58]:
l = set([1,2])
s.update(l) # 使用update像集合里面添加元素

In [57]:
s

{1, 2}

In [59]:
# 开始尝试探索新入网，新离网用户

data.head()

,imsi_,update_time_,lac_,cellid_,status_,ap_type_,device_id_,eventid_
update_time_,,,,,,,,
2019-05-08 20:09:29,460110413044304,2019-05-08 20:09:29,0,0,拒绝,FDD-LTE,DX-SZSC001,Normal Lau
2019-05-08 20:09:41,460110413042577,2019-05-08 20:09:41,0,0,拒绝,FDD-LTE,DX-SZSC001,Normal Lau
2019-05-08 20:10:14,460110425023300,2019-05-08 20:10:14,0,0,拒绝,FDD-LTE,DX-SZSC001,Normal Lau
2019-05-08 20:10:40,460110714141865,2019-05-08 20:10:40,0,0,拒绝,FDD-LTE,DX-SZSC001,Normal Lau
2019-05-08 20:08:55,460016692088409,2019-05-08 20:08:55,0,0,拒绝,FDD-LTE,LT-SZSC001,Normal Lau


In [65]:
window = (data.index.max().day - data.index.min().day) // 5

In [66]:
window # 设置时间窗口

3

In [106]:
datetime.datetime(2019,5,8).

'2019-05-08 00:00:00'

In [104]:
data.loc['20190508':'20190510']

,imsi_,update_time_,lac_,cellid_,status_,ap_type_,device_id_,eventid_
update_time_,,,,,,,,
2019-05-08 20:09:29,460110413044304,2019-05-08 20:09:29,0,0,拒绝,FDD-LTE,DX-SZSC001,Normal Lau
2019-05-08 20:09:41,460110413042577,2019-05-08 20:09:41,0,0,拒绝,FDD-LTE,DX-SZSC001,Normal Lau
2019-05-08 20:10:14,460110425023300,2019-05-08 20:10:14,0,0,拒绝,FDD-LTE,DX-SZSC001,Normal Lau
2019-05-08 20:10:40,460110714141865,2019-05-08 20:10:40,0,0,拒绝,FDD-LTE,DX-SZSC001,Normal Lau
2019-05-08 20:08:55,460016692088409,2019-05-08 20:08:55,0,0,拒绝,FDD-LTE,LT-SZSC001,Normal Lau
...,...,...,...,...,...,...,...,...
2019-05-10 23:59:04,460013010514502,2019-05-10 23:59:04,0,0,拒绝,FDD-LTE,LT-SZSC001,Normal Lau
2019-05-10 23:59:06,460016607527914,2019-05-10 23:59:06,0,0,拒绝,FDD-LTE,LT-SZSC001,Normal Lau
2019-05-10 23:59:08,460015442027265,2019-05-10 23:59:08,0,0,拒绝,FDD-LTE,LT-SZSC001,Normal Lau


In [76]:
date(2020,5,4) + time(27)

ValueError: hour must be in 0..23

In [83]:
import datetime

In [95]:
year.min()

2019

In [91]:
datetime.datetime(2020,2,23) + datetime.timedelta(days=9) # 这样可以实现时间相加，且不用担心月份会溢出的问题

datetime.datetime(2020, 3, 3, 0, 0)

In [86]:
datetime.date.today() + datetime.timedelta(days=3)

datetime.date(2020, 9, 16)

In [116]:
# 初步调试
year = data.index.year.min()
month = data.index.month.min()
day = data.index.day.min() # 本来不加min返回的是所有年份
time1 = datetime.datetime(year, month, day)

time_window = datetime.timedelta(days=window)

time2 = (time1 + time_window)
# 将时间类型数据转换成字符串
stime1, stime2 = time1.strftime(format='%Y-%m-%d %H:%M:%S'), time2.strftime(format='%Y-%m-%d %H:%M:%S')

data.imsi_.loc[stime1:stime2]

update_time_
2019-05-08 20:09:29    460110413044304
2019-05-08 20:09:41    460110413042577
2019-05-08 20:10:14    460110425023300
2019-05-08 20:10:40    460110714141865
2019-05-08 20:08:55    460016692088409
                            ...       
2019-05-10 23:59:04    460013010514502
2019-05-10 23:59:06    460016607527914
2019-05-10 23:59:08    460015442027265
2019-05-10 23:59:13    460110410817264
2019-05-10 23:59:25    460016756520273
Name: imsi_, Length: 38470, dtype: int64

In [118]:
base_user = set(data.imsi_.loc[stime1:stime2])

In [114]:
# 这里假设0表示新入网，1表示新离网，2表示常出现用户
d = dict(zip(set(data.imsi_), [0]*data.shape[0]))
data['statues'] = data.imsi_.map(pd.Series(d))

In [115]:
data.head()

,imsi_,update_time_,lac_,cellid_,status_,ap_type_,device_id_,eventid_,statues
update_time_,,,,,,,,,
2019-05-08 20:09:29,460110413044304,2019-05-08 20:09:29,0,0,拒绝,FDD-LTE,DX-SZSC001,Normal Lau,0
2019-05-08 20:09:41,460110413042577,2019-05-08 20:09:41,0,0,拒绝,FDD-LTE,DX-SZSC001,Normal Lau,0
2019-05-08 20:10:14,460110425023300,2019-05-08 20:10:14,0,0,拒绝,FDD-LTE,DX-SZSC001,Normal Lau,0
2019-05-08 20:10:40,460110714141865,2019-05-08 20:10:40,0,0,拒绝,FDD-LTE,DX-SZSC001,Normal Lau,0
2019-05-08 20:08:55,460016692088409,2019-05-08 20:08:55,0,0,拒绝,FDD-LTE,LT-SZSC001,Normal Lau,0


In [123]:
time1, time2 = time2, time1 + time_window
stime1, stime2 = time1.strftime(format='%Y-%m-%d %H:%M:%S'), time2.strftime(format='%Y-%m-%d %H:%M:%S')
user = set(data.imsi_.loc[stime1:stime2])
new_leaver = base_user - user # 注意集合运算的顺序 1 新离网
permanent = base_user & user # 2 常驻用户
new_commer = user - base_user # 0 新入网

In [124]:
# 开始更新用户状态
base_user.update(user) # 先增加基础用户，所有出现的用户都要加入进去
leaver_code = dict(zip(new_leaver, [1]*len(new_leaver)))
permanent_code = dict(zip(permanent, [2]*len(permanent)))
commer_code = dict(zip(new_commer, [0]*len(new_commer)))

In [125]:
data['statues'] = data.imsi_.map(leaver_code)

In [129]:
data['statues'] # 发现不匹配的会被认定为空值

update_time_
2019-05-08 20:09:29    NaN
2019-05-08 20:09:41    1.0
2019-05-08 20:10:14    1.0
2019-05-08 20:10:40    NaN
2019-05-08 20:08:55    NaN
                      ... 
2019-05-24 22:38:32    NaN
2019-05-24 22:38:34    NaN
2019-05-24 22:38:36    1.0
2019-05-24 22:39:00    NaN
2019-05-24 22:39:04    NaN
Name: statues, Length: 338898, dtype: float64

解决办法，都组合成一个字典

In [138]:
coder = {}
coder.update([leaver_code, permanent_code, commer_code])

ValueError: dictionary update sequence element #0 has length 9248; 2 is required

In [141]:
for each in [leaver_code, permanent_code, commer_code]:
    coder.update(each)

{460069107007489: 1,
 460017051631619: 1,
 460110411595787: 1,
 460012348604444: 1,
 460078194425886: 1,
 460016087629858: 1,
 460007444250659: 1,
 460090842120226: 1,
 460015316107305: 1,
 460016291610665: 1,
 460023116398641: 1,
 460071791657011: 1,
 460110411399223: 1,
 460007392673852: 1,
 460071791657020: 1,
 460110410776637: 1,
 460002601893952: 1,
 460021606711373: 1,
 460022200664143: 1,
 460009190391888: 1,
 460013573505104: 1,
 460006851838031: 1,
 460016637509718: 1,
 460045960904799: 1,
 460018862915684: 1,
 460000500318309: 1,
 460012048613480: 1,
 460078237253736: 1,
 460110412841067: 1,
 460000970178667: 1,
 460017912316013: 1,
 460018678202482: 1,
 460016667623541: 1,
 460015440035959: 1,
 460110411858040: 1,
 460018486607999: 1,
 460016840016001: 1,
 460002321563780: 1,
 460010190504071: 1,
 460002306850951: 1,
 460010420502665: 1,
 460093020012682: 1,
 460012268617866: 1,
 460110415069326: 1,
 460017227628687: 1,
 460110714175631: 1,
 460110959214737: 1,
 460000146620

In [143]:
data['statues'] = data.imsi_.map(coder)

In [145]:
data['statues'].value_counts()

2.0    166220
0.0     59194
1.0     26460
Name: statues, dtype: int64

In [147]:
data['statues'].isnull().mean() # 为什么没有全部取到？因为时间周期没有遍历完

0.2567852274135581

In [148]:
t1 = {1,2,3}
t2 = {2,3,4,5}

In [149]:
t1 - t2 # 新离网

{1}

In [150]:
t2 - t1 # 新入网

{4, 5}

In [151]:
t1 & t2 # 常驻用户

{2, 3}

In [152]:
# 所以应该建立个时间列表，然后遍历这些时间，将时间切片好

In [162]:
data.head()

,imsi_,update_time_,lac_,cellid_,status_,ap_type_,device_id_,eventid_
update_time_,,,,,,,,
2019-05-08 20:09:29,460110413044304,2019-05-08 20:09:29,0,0,拒绝,FDD-LTE,DX-SZSC001,Normal Lau
2019-05-08 20:09:41,460110413042577,2019-05-08 20:09:41,0,0,拒绝,FDD-LTE,DX-SZSC001,Normal Lau
2019-05-08 20:10:14,460110425023300,2019-05-08 20:10:14,0,0,拒绝,FDD-LTE,DX-SZSC001,Normal Lau
2019-05-08 20:10:40,460110714141865,2019-05-08 20:10:40,0,0,拒绝,FDD-LTE,DX-SZSC001,Normal Lau
2019-05-08 20:08:55,460016692088409,2019-05-08 20:08:55,0,0,拒绝,FDD-LTE,LT-SZSC001,Normal Lau


In [175]:
t = list(set(data.index.day))
t

[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]

In [180]:
# 这样就切片好了
# 但如果是不同月份的话，还需要改进，先实现功能再说
length = len(t)
res = []
i = 0
while i + window < length:
    res.append(t[i:i+window])
    i = i + window
res.append(t[i:])
res

[[8, 9, 10], [11, 12, 13], [14, 15, 16], [17, 18, 19], [20, 21, 22], [23, 24]]

In [196]:
# 初步调试
year = data.index.year.min()
month = data.index.month.min()
day = data.index.day.min() # 本来不加min返回的是所有年份
time1 = datetime.datetime(year, month, day)

time_window = datetime.timedelta(days=window)

time2 = (time1 + time_window)
# 将时间类型数据转换成字符串
stime1, stime2 = time1.strftime(format='%Y-%m-%d %H:%M:%S'), time2.strftime(format='%Y-%m-%d %H:%M:%S')

# imsi = data.imsi_.loc[stime1:stime2]

base_user = set(data.imsi_.loc[stime1:stime2])

d = {}
d.update(dict(zip(base_user, [0]*len(base_user))))

while time1.day + window < data.index.day.max(): # 不用担心超过时间区间
    time1, time2 = time2, time2 + time_window
    print(time1.day, time2.day)
    stime1, stime2 = time1.strftime(format='%Y-%m-%d %H:%M:%S'), time2.strftime(format='%Y-%m-%d %H:%M:%S')
    user = set(data.imsi_.loc[stime1:stime2])
    d.update(dict(zip(user, [0]*len(user))))

11 14
14 17
17 20
20 23
23 26


In [191]:
len()

Timestamp('2019-05-24 22:39:04')

In [185]:
time1.day

11

In [199]:
len(d)

92509

In [188]:
len(set(data.index)) # 为什么所有用户id没对上

277489

In [202]:
# 初步调试
year = data.index.year.min()
month = data.index.month.min()
day = data.index.day.min() # 本来不加min返回的是所有年份
time1 = datetime.datetime(year, month, day)

time_window = datetime.timedelta(days=window)

time2 = (time1 + time_window)
# 将时间类型数据转换成字符串
stime1, stime2 = time1.strftime(format='%Y-%m-%d %H:%M:%S'), time2.strftime(format='%Y-%m-%d %H:%M:%S')

data.imsi_.loc[stime1:stime2]

base_user = set(data.imsi_.loc[stime1:stime2])

time1, time2 = time2, time1 + time_window
stime1, stime2 = time1.strftime(format='%Y-%m-%d %H:%M:%S'), time2.strftime(format='%Y-%m-%d %H:%M:%S')
user = set(data.imsi_.loc[stime1:stime2])
new_leaver = base_user - user # 注意集合运算的顺序 1 新离网
permanent = base_user & user # 2 常驻用户
new_commer = user - base_user # 0 新入网

In [203]:
len(new_commer)+len(new_leaver)+len(permanent)

14134

In [204]:
len(base_user) + len(user) # 两个数据都对的到，说明可以的e，接下来时间滑动就好了

14134

In [303]:
# 完整调试
year = data.index.year.min()
month = data.index.month.min()
day = data.index.day.min() # 本来不加min返回的是所有年份
time1 = datetime.datetime(year, month, day)

time_window = datetime.timedelta(days=window) # 这样可以直接和时间数据相加

time2 = (time1 + time_window)
# 将时间类型数据转换成字符串
stime1, stime2 = time1.strftime(format='%Y-%m-%d %H:%M:%S'), time2.strftime(format='%Y-%m-%d %H:%M:%S')

# data.imsi_.loc[stime1:stime2]
d = {}
base_user = set(data.imsi_.loc[stime1:stime2])A
while time1.day + window <= data.index.day.max():
    time1, time2 = time2, time2 + time_window
    stime1, stime2 = time1.strftime(format='%Y-%m-%d %H:%M:%S'), time2.strftime(format='%Y-%m-%d %H:%M:%S')
    user = set(data.imsi_.loc[stime1:stime2]) # 新用户
    new_leaver = base_user - user # 注意集合运算的顺序 1 新离网
    permanent = base_user & user # 2 常驻用户
    new_commer = user - base_user # 0 新入网
    base_user.update(user)
    d.update(dict(zip(new_leaver, len(new_leaver)*[1])))
    d.update(dict(zip(new_commer, len(new_commer)*[0])))
    d.update(dict(zip(permanent, len(permanent)*[2])))

In [304]:
data['status'] = data.imsi_.map(d)

In [306]:
data.status.isnull().sum()

0

In [209]:
data['status'] = data.imsi_.map(d)

In [215]:
data.status.count() + data.status.isnull().count() # 怎么缺了这么多数据，会不会是因为时间的区间开闭出现问题

664908

In [214]:
len(set(data.index))

277489

In [216]:
data.index.min()

Timestamp('2019-05-08 20:08:55')

In [249]:
s1 = set(data.imsi_['2019-5-8':'2019-5-11'])

In [248]:
s2 = set(data.imsi_['2019-5-12':'2019-5-14'])

In [266]:
len(set(data.imsi_['2019-5-8':'2019-5-14']))

42338

In [251]:
s1.update(s2)

In [252]:
len(s1)

42338

In [267]:
# 检查下数据为什么少了这么多。可以发现在第二个循环开始缺少数据的

t1 = s1 - s2
t2 = s2 - s1
t3 = s1 & s2
len(t1) + len(t2) + len(t3)

42338

In [268]:
s3 = set(data.imsi_['2019-5-15':'2019-5-18'])

In [269]:
len(s3)

28702

In [278]:
s = set()

In [279]:
s.update(s1)
s.update(s2)

In [280]:
len(s)

42338

In [281]:
t1 = s1 - s3
t2 = s3 - s1
t3 = s1 & s3
len(t1) + len(t2) + len(t3)

61748

In [273]:
s.update(s3)

In [275]:
len(s)

61748

In [300]:
len(set(data.imsi_[:'2019-5-10']))

14134

In [287]:
data.imsi_.loc[stime1:stime2]

Series([], Name: imsi_, dtype: int64)

In [288]:
stime1

'2019-05-24 00:00:00'

In [322]:
data.status.value_counts()

2    187829
1    138181
0     12888
Name: status, dtype: int64

In [323]:
data.head()

,imsi_,update_time_,lac_,cellid_,status_,ap_type_,device_id_,eventid_,status
update_time_,,,,,,,,,
2019-05-08 20:09:29,460110413044304,2019-05-08 20:09:29,0,0,拒绝,FDD-LTE,DX-SZSC001,Normal Lau,2
2019-05-08 20:09:41,460110413042577,2019-05-08 20:09:41,0,0,拒绝,FDD-LTE,DX-SZSC001,Normal Lau,1
2019-05-08 20:10:14,460110425023300,2019-05-08 20:10:14,0,0,拒绝,FDD-LTE,DX-SZSC001,Normal Lau,1
2019-05-08 20:10:40,460110714141865,2019-05-08 20:10:40,0,0,拒绝,FDD-LTE,DX-SZSC001,Normal Lau,2
2019-05-08 20:08:55,460016692088409,2019-05-08 20:08:55,0,0,拒绝,FDD-LTE,LT-SZSC001,Normal Lau,2


In [324]:
d

{460069107007489: 1,
 460017051631619: 1,
 460110411595787: 1,
 460012348604444: 1,
 460078194425886: 1,
 460016087629858: 1,
 460007444250659: 2,
 460090842120226: 1,
 460015316107305: 1,
 460016291610665: 1,
 460023116398641: 1,
 460071791657011: 1,
 460110411399223: 1,
 460007392673852: 1,
 460071791657020: 2,
 460110410776637: 1,
 460002601893952: 2,
 460021606711373: 1,
 460022200664143: 1,
 460009190391888: 2,
 460013573505104: 1,
 460006851838031: 1,
 460016637509718: 1,
 460045960904799: 1,
 460018862915684: 1,
 460000500318309: 1,
 460012048613480: 1,
 460078237253736: 1,
 460110412841067: 1,
 460000970178667: 1,
 460017912316013: 1,
 460018678202482: 1,
 460016667623541: 1,
 460015440035959: 1,
 460110411858040: 1,
 460018486607999: 2,
 460016840016001: 1,
 460002321563780: 2,
 460010190504071: 1,
 460002306850951: 1,
 460010420502665: 1,
 460093020012682: 1,
 460012268617866: 1,
 460110415069326: 1,
 460017227628687: 1,
 460110714175631: 1,
 460110959214737: 1,
 460000146620